# Data Importing And Cleaning

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AFFL_hAmhLZhWVLGEXzRlOMXaTSi_wVog_pclYrt6BouMwZTLgBMy0
Mounted at /content/gdrive


In [2]:
import pandas as pd
df = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/Datasets/Incidents_service.xlsx')
df.shape

(141712, 25)

In [3]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## 1.Feature Selection

In [4]:
df_new = df[[ 'count_updated', 'opened_by', 'Support_group', 'category_ID', 'ID_status','impact']]
df_new.shape

(141712, 6)

In [5]:
df_new = df_new[df_new.ID_status != -100]
df_new.ID_status.value_counts()

Active                38716
New                   36407
Resolved              25751
Closed                24985
Awaiting User Info    14642
Awaiting Vendor         707
Awaiting Problem        461
Awaiting Evidence        38
Name: ID_status, dtype: int64

In [6]:
df_copy = df_new
df_new.head()
print(df_copy.shape)
print(df_new.shape)

(141707, 6)
(141707, 6)


In [7]:
df_new = pd.get_dummies(df_new, columns=['Support_group','category_ID','ID_status','opened_by'], drop_first=True)
print(df_new.shape)
df_new.head()

(141707, 349)


,count_updated,impact,Support_group_Group 11,Support_group_Group 12,Support_group_Group 13,Support_group_Group 14,Support_group_Group 15,Support_group_Group 16,Support_group_Group 17,Support_group_Group 18,Support_group_Group 19,Support_group_Group 2,Support_group_Group 20,Support_group_Group 21,Support_group_Group 22,Support_group_Group 23,Support_group_Group 24,Support_group_Group 25,Support_group_Group 26,Support_group_Group 27,Support_group_Group 28,Support_group_Group 29,Support_group_Group 3,Support_group_Group 30,Support_group_Group 31,Support_group_Group 32,Support_group_Group 33,Support_group_Group 34,Support_group_Group 35,Support_group_Group 36,Support_group_Group 37,Support_group_Group 38,Support_group_Group 39,Support_group_Group 4,Support_group_Group 41,Support_group_Group 43,Support_group_Group 44,Support_group_Group 45,Support_group_Group 46,Support_group_Group 47,...,opened_by_Opened by 503,opened_by_Opened by 505,opened_by_Opened by 508,opened_by_Opened by 512,opened_by_Opened by 515,opened_by_Opened by 517,opened_by_Opened by 519,opened_by_Opened by 52,opened_by_Opened by 520,opened_by_Opened by 524,opened_by_Opened by 528,opened_by_Opened by 530,opened_by_Opened by 533,opened_by_Opened by 535,opened_by_Opened by 536,opened_by_Opened by 538,opened_by_Opened by 54,opened_by_Opened by 55,opened_by_Opened by 58,opened_by_Opened by 59,opened_by_Opened by 60,opened_by_Opened by 62,opened_by_Opened by 66,opened_by_Opened by 67,opened_by_Opened by 68,opened_by_Opened by 7,opened_by_Opened by 70,opened_by_Opened by 72,opened_by_Opened by 73,opened_by_Opened by 74,opened_by_Opened by 75,opened_by_Opened by 77,opened_by_Opened by 79,opened_by_Opened by 8,opened_by_Opened by 80,opened_by_Opened by 88,opened_by_Opened by 9,opened_by_Opened by 90,opened_by_Opened by 91,opened_by_Opened by 94
0,0,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,3,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,4,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,1,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,2 - Medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
impact = df_new['impact']
df_new.drop(columns = 'impact', inplace = True)
impact.shape

(141707,)

## 2. Scaling

In [9]:
from sklearn.preprocessing import MinMaxScaler
mx_scaler = MinMaxScaler()
final_df2 = mx_scaler.fit_transform(df_new)
final_df2 = pd.DataFrame(final_df2, columns=df_new.columns)
print(final_df2.head())

   count_updated  ...  opened_by_Opened by  94
0       0.000000  ...                      0.0
1       0.023256  ...                      0.0
2       0.031008  ...                      0.0
3       0.007752  ...                      0.0
4       0.015504  ...                      0.0

[5 rows x 348 columns]


# Dimensionality Reduction PCA

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
pipe = Pipeline([
('scaler', StandardScaler()),
('reducer', PCA( n_components=10 ))])
pca = pipe.fit_transform(final_df2)
print(pca)

[[ 1.21570842  0.04839237 -3.17524165 ...  1.03437783 -0.53092245
   0.36926973]
 [ 1.56492131  0.29255212 -3.1292663  ...  0.76598507 -0.6543455
   0.84133664]
 [ 1.65978736  0.25109888 -3.11496507 ...  0.811982   -0.6689022
   0.86872511]
 ...
 [-4.35354434 -1.09327622  5.42559575 ... -2.34466715 -6.95576163
   7.18527822]
 [-4.35354434 -1.09327622  5.42559575 ... -2.34466715 -6.95576163
   7.18527822]
 [ 0.66566607  0.35899169 -2.1584049  ... -0.56056368  0.69193704
  -0.93469329]]


In [11]:
final_df = pd.DataFrame(pca)
final_df['impact'] = df_copy['impact']
final_df.head()

,0,1,2,3,4,5,6,7,8,9,impact
0,1.215708,0.048392,-3.175242,-0.264121,4.263253,-0.276858,-0.672087,1.034378,-0.530922,0.369270,2 - Medium
1,1.564921,0.292552,-3.129266,-0.271465,4.210472,-0.698070,-0.528848,0.765985,-0.654345,0.841337,2 - Medium
2,1.659787,0.251099,-3.114965,-0.241700,4.224143,-0.833113,-0.460184,0.811982,-0.668902,0.868725,2 - Medium
3,0.008328,-0.294165,0.419040,-0.001579,0.517419,0.481460,-0.146617,0.739335,-0.784054,-0.071898,2 - Medium
4,0.055095,-0.308219,0.428912,-0.003547,0.512783,0.471317,-0.139846,0.738365,-0.790354,-0.067785,2 - Medium


# Train Test Split

In [12]:
from sklearn.model_selection import train_test_split
X = final_df.drop(columns = 'impact')
y = impact

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101, stratify= y)

In [14]:
print(y_train.value_counts())
print(y_test.value_counts())

2 - Medium    94030
3 - Low        2720
1 - High       2444
Name: impact, dtype: int64
2 - Medium    40300
3 - Low        1166
1 - High       1047
Name: impact, dtype: int64


# Algorithms

## 1. Decision Tree:

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
dt = DecisionTreeClassifier(random_state=101)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9435231576223744
[[  429   601    17]
 [  637 39248   415]
 [   14   717   435]]
              precision    recall  f1-score   support

    1 - High       0.40      0.41      0.40      1047
  2 - Medium       0.97      0.97      0.97     40300
     3 - Low       0.50      0.37      0.43      1166

    accuracy                           0.94     42513
   macro avg       0.62      0.59      0.60     42513
weighted avg       0.94      0.94      0.94     42513



## 2. Random Forest:

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators= 500, random_state= 101)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9547432550043516
[[  341   695    11]
 [  228 39860   212]
 [    9   769   388]]
              precision    recall  f1-score   support

    1 - High       0.59      0.33      0.42      1047
  2 - Medium       0.96      0.99      0.98     40300
     3 - Low       0.64      0.33      0.44      1166

    accuracy                           0.95     42513
   macro avg       0.73      0.55      0.61     42513
weighted avg       0.95      0.95      0.95     42513



## 3.Support Vector Classifier

In [17]:
from sklearn.svm import SVC
svc_clf = SVC()
svc_clf.fit(X_train,y_train)
y_pred = svc_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9513090113612307
[[   31  1015     1]
 [    3 40282    15]
 [    7  1029   130]]
              precision    recall  f1-score   support

    1 - High       0.76      0.03      0.06      1047
  2 - Medium       0.95      1.00      0.98     40300
     3 - Low       0.89      0.11      0.20      1166

    accuracy                           0.95     42513
   macro avg       0.87      0.38      0.41     42513
weighted avg       0.95      0.95      0.93     42513



## 4. Gausian NB

In [18]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train,y_train)
y_pred = nb.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8958906687366217
[[   51   926    70]
 [  545 37884  1871]
 [   24   990   152]]
              precision    recall  f1-score   support

    1 - High       0.08      0.05      0.06      1047
  2 - Medium       0.95      0.94      0.95     40300
     3 - Low       0.07      0.13      0.09      1166

    accuracy                           0.90     42513
   macro avg       0.37      0.37      0.37     42513
weighted avg       0.91      0.90      0.90     42513



## 5. KNN

In [19]:
from sklearn.neighbors  import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9520617222967093
[[  447   584    16]
 [  341 39515   444]
 [   20   633   513]]
              precision    recall  f1-score   support

    1 - High       0.55      0.43      0.48      1047
  2 - Medium       0.97      0.98      0.98     40300
     3 - Low       0.53      0.44      0.48      1166

    accuracy                           0.95     42513
   macro avg       0.68      0.62      0.65     42513
weighted avg       0.95      0.95      0.95     42513



# Meta Algorithms

## 1. Voting Classifier

In [20]:
from sklearn.ensemble import VotingClassifier
nb = GaussianNB()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier(random_state=101)
# Define a list called classifier that contains the tuples (classifier_name, classifier)
classifiers = [('GaussianNB', nb),
('K Nearest Neighbours', knn),
('Classification Tree', dt)]
# Iterate over the defined list of tuples containing the classifiers
for clf_name, clf in classifiers:
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print('{:s} : {:.3f}'.format(clf_name, accuracy_score(y_test, y_pred)))

vc = VotingClassifier(estimators=classifiers)
vc.fit(X_train, y_train)
y_pred = vc.predict(X_test)
print('Voting Classifier: {:.3f}'.format(accuracy_score(y_test, y_pred)))

GaussianNB : 0.896
K Nearest Neighbours : 0.954
Classification Tree : 0.944
Voting Classifier: 0.954


## 2. Bagging Classifier

In [21]:
from sklearn.ensemble import BaggingClassifier
dt = DecisionTreeClassifier(random_state=101)
bc = BaggingClassifier(base_estimator=dt, n_estimators=300, n_jobs=-1)
bc.fit(X_train, y_train)
y_pred = bc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of Bagging Classifier: {:.3f}'.format(accuracy))

Accuracy of Bagging Classifier: 0.954


## 3. AdaBoost Classifier

In [22]:
from sklearn.ensemble import AdaBoostClassifier
dt = DecisionTreeClassifier(random_state=101)
abc = AdaBoostClassifier(base_estimator=dt, n_estimators=300)
abc.fit(X_train, y_train)
y_pred = bc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of Bagging Classifier: {:.3f}'.format(accuracy))

Accuracy of Bagging Classifier: 0.954


## 4. XGBoost Classifier

In [23]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of Bagging Classifier: {:.3f}'.format(accuracy))

Accuracy of Bagging Classifier: 0.951
